In [46]:
import os
import pandas as pd
import dotenv
dotenv.load_dotenv(dotenv.find_dotenv())
import psycopg2
import time
from sqlalchemy import create_engine
import json
from unidecode import unidecode
import re
import numpy as np
from datetime import datetime

In [47]:
def connect_db():
    try:
        alchemy_url = "postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}".format(
            user=os.getenv('login_sql_tech'),
            password=os.getenv('senha_sql_tech'),
            host=os.getenv('host_sql_tech'),
            port=os.getenv('port_sql_tech'),
            database=os.getenv('database_sql_tech')
        )
        return create_engine(alchemy_url)
    except:
        print("Erro ao conectar ao banco de dados")
        return False

In [48]:
engine = connect_db()

In [63]:
id_contrato = 4422

In [64]:
dataFrame_infos = pd.read_sql_query("SELECT * FROM clientes WHERE id_contrato = {}".format(id_contrato), engine)

In [65]:
valor_contrato = dataFrame_infos['valor_contrato'][0]
data_contrato = dataFrame_infos['data_contrato'][0]

print('Valor contrato:',valor_contrato)
print('Data contrato:',data_contrato)

Valor contrato: 83607.0
Data contrato: 2022-04-05


In [67]:
dataFrame_parcelas = pd.read_sql_query(
    "SELECT * FROM parcelas WHERE id_contrato = {} and (tipo = 'Parcela' or tipo = 'Entrada') ORDER BY sequencia ASC".format(id_contrato),
    con = engine
)


total_dias_atraso = 0
valor_pago = 0
total_parcelas_pagas_com_atrasadas = 0
total_parcelas_ = 0

for row in dataFrame_parcelas.itertuples():
    # print(type(row.vencimento))
    total_dias = total_dias + row.dias_atraso
    valor_pago = valor_pago + row.valor_pago
    if(row.dias_atraso > 0):
        total_parcelas_pagas_com_atrasadas = total_parcelas_pagas_com_atrasadas + 1
    if(row.vencimento > datetime.date(datetime.now())):
        print('achou')
        break
    if(row.sequencia.split('/')[0] == '012'):
        break
    total_parcelas_ = total_parcelas_ + 1

print('Total de dias com atraso:',total_dias)
print('razao (valor pago/total a pagar):', valor_pago/valor_contrato)
print('Total de parcelas pagas com atraso:',total_parcelas_pagas_com_atrasadas)
print('Total de parcelas:',total_parcelas_)

    

Total de dias com atraso: 8.0
razao (valor pago/total a pagar): 1.0000008372504696
Total de parcelas pagas com atraso: 1
Total de parcelas: 2
